In [2]:
import re
import numpy as np
import pandas as pd

from collections import Counter


# Preprocess the data

In [5]:
%cd /content/drive/MyDrive/Colab Notebooks/nlp/apps/autocorrect

/content/drive/MyDrive/Colab Notebooks/nlp/apps/autocorrect


In [14]:
path = 'data/sherlock_novels.txt'

In [12]:
def preprocessing(filename):
    """
    Takes a txt file and returns the words in the file
    Args:
        filename: path to the file
    return:
        words: list containing the words in the file
    """
    words = []
    with open(filename) as f:
        text = f.read()

    # \w: Returns a match where the string contains any word characters 
    # +: One or more occurrences
    words = re.findall(r'\w+', text.lower())
    return words

#sherlock_words = preprocessing('data/sherlock_novels.txt')
#sherlock_words[:10]
#len(sherlock_words)


In [13]:
def get_count(words):
    """
    Returns a dict with the count of each word in a corpus
    Args:
        words: list
    return:
        word_count: dict
    """
    word_count = Counter(words)

    return word_count

sherlock = preprocessing(path)
word_count = get_count(sherlock)
w
